In [ ]:
# all_slow

In [ ]:
from wwf.utils import state_versions

state_versions(['wwf', 'fastai', 'fastcore', 'nbdev'])


---
This article is also a Jupyter Notebook available to be run from the top down. There
will be code snippets that you can then run in any environment.

Below are the versions of `wwf`, `fastai`, `fastcore`, and `nbdev` currently running at the time of writing this:
* `wwf`: 0.0.13 
* `fastai`: 2.2.5 
* `fastcore`: 1.3.19 
* `nbdev`: 1.1.12 
---

In [ ]:
from fastai.text.all import *
from utils import *

In [ ]:
dls = build_dls()

In [ ]:
dls.show_batch()

In [ ]:
# From goralpl's kernel here: https://github.com/goralpl/learning_fastai/blob/master/seq2seq_fastai_datablocks_custom_model.ipynb

INPUT_DIM = len(dls.train.vocab[0])
OUTPUT_DIM = len(dls.train.vocab[0])
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

In [ ]:
net = Seq2Seq(enc,dec,'cpu')

In [ ]:
doc(net.forward)

In [ ]:
learn = Learner(dls, net, loss_func=CrossEntropyLossFlat())

In [ ]:
class TeacherForcingCallback(Callback):
    "A Callback that sends targ to model and adjusts teacher forcing ratio"
    def __init__(self, teacher_forcing_ratio:int=0.5): self.teacher_forcing_ratio = teacher_forcing_ratio
    def before_batch(self):
        x,y = self.x, self.y
        self.learn.xb = (x,y,self.teacher_forcing_ratio)

In [ ]:
class StudentForcingRatio(Callback):

    def before_batch(self):
        self.teacher_forcing_ratio

In [ ]:
learn.fit(1, cbs=[TeacherForcingCallback])

epoch,train_loss,valid_loss,time
0,3.466535,3.465464,00:00
